In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, ".");

In [ ]:
using Images
using VideoIO: openvideo
using Unitful

In [ ]:
import Cornercam
reload("Cornercam")

In [ ]:
video = VideoIO.openvideo("DSCF7139.MOV")
source = video

In [ ]:
using BenchmarkTools

In [ ]:
blur = Cornercam.OffGridBlur{6}(3.0)
Cornercam.sample_blurred.((frame,), collect(CartesianRange((500, 500))), blur)

In [ ]:
Cornercam.weights(blur, 10.51)

In [ ]:
@benchmark Cornercam.sample_blurred($(RGB{Float32}.(frame)), (10, 10), $blur)

In [ ]:
background = mean((RGB{Float32}.(im) for im in Cornercam.frames_between(video, (10u"s", 12u"s"))))

In [ ]:
original_corners = [[236, 404], 
    [230, 920], 
    [719, 948], 
    [696, 340]]

H = inv(Cornercam.rectify(original_corners))

params = Cornercam.Params()
cam = Cornercam.CornerCamera(source, H, background, params, √(2.7));

In [ ]:
cam.homography.(cam.params.samples)[end-1:end]

In [ ]:
Cornercam.show_samples(cam)

In [ ]:
@time trace = Cornercam.trace(cam, (4u"s", 20u"s"), blur, 10/u"s")

In [ ]:
Cornercam.imnormal(trace)

In [ ]:
Cornercam.imnormal(trace[0.1..π/2, 5u"s"..10u"s"])